In [ ]:
# Import the required libraries and open the connection to Mongo

import collections
import datetime
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (15, 15) # Reset the base size of figures so they're large enough to be useful.

import pandas as pd
import scipy.stats

import folium

import pymongo

In [ ]:
# Open a connection to the Mongo server, open the accidents database and name the collections of accidents and labels
client = pymongo.MongoClient('mongodb://localhost:27351/')

db = client.accidents
accidents = db.accidents
labels = db.labels
roads = db.roads

In [ ]:
# Load the expanded names of keys and human-readable codes into memory
expanded_name = collections.defaultdict(str)
for e in labels.find({'expanded': {"$exists": True}}):
    expanded_name[e['label']] = e['expanded']
    
label_of = collections.defaultdict(str)
for l in labels.find({'codes': {"$exists": True}}):
    for c in l['codes']:
        try:
            label_of[l['label'], int(c)] = l['codes'][c]
        except ValueError: 
            label_of[l['label'], c] = l['codes'][c]

In [ ]:
# def make_map(location, width=800, height=800, zoom_start=9):
#     m = folium.Map(location=location, width=width, height=height, zoom_start=zoom_start)
#     m.save('folium-map-' + uuid.uuid4().hex + '.html')
#     m.render_iframe = True
#     return m

In [ ]:
def add_accidents_markers(the_map, query, number_of_sides=5, fill_color='#769d96', 
                     radius=5, rotation=54):
    for a in accidents.find(query, 
                        ['loc.coordinates']):
        folium.RegularPolygonMarker(location=[a['loc']['coordinates'][1], a['loc']['coordinates'][0]], 
                     number_of_sides=number_of_sides, radius=radius, rotation=rotation,
                                   fill_color=fill_color).add_to(the_map)   

In [ ]:
def map_centre_from_points(query):
    latlons = list(zip(*[(a['loc']['coordinates'][1], 
                      a['loc']['coordinates'][0]) 
                    for a in accidents.find(query, 
                        ['loc.coordinates'])]))
    max_lat = max(latlons[0])
    max_lon = max(latlons[1])
    min_lat = min(latlons[0])
    min_lon = min(latlons[1]) 
    return [min_lat + (max_lat - min_lat) / 2,
            min_lon + (max_lon - min_lon) / 2]

### Activity 1

In [ ]:
query = {'$or': 
         [{'1st_Road_Class': 1},
          {'2nd_Road_Class': 1},
          {'1st_Road_Class': 2},
          {'2nd_Road_Class': 2}],
         'Accident_Severity': {'$in': [1, 2]}}

m = folium.Map([55, -3], zoom_start=6)    
add_accidents_markers(m, query)
m

### Activity 2

In [ ]:
# Find the codes of the local authorities
[(code, label, label_of[label, code]) for label, code in label_of 
 if 'Cornwall' in label_of[label, code]]

In [ ]:
query = {'Local_Authority_(Highway)': 'E06000052'}

map_centre = map_centre_from_points(query)

m = folium.Map(map_centre, zoom_start=9)
add_accidents_markers(m, query)
m

### Activity 3

I'm basing this on the town of Buckingham.

In [ ]:
region = {'loc': 
          {'$nearSphere':
           {'$geometry': 
            {'type': 'Point', 
             'coordinates': [-0.987645, 51.999326]},
            '$maxDistance': 10000}}}

map_centre = map_centre_from_points(region)

m = folium.Map(map_centre, zoom_start=11)    
add_accidents_markers(m, region)
m

In [ ]:
buckingham = {'type': 'Polygon',
              'coordinates': [[[-1.212961, 52.133015],
                               [-0.762329, 52.133015],
                               [-0.762329, 51.865637],
                               [-1.212961, 51.865637],
                               [-1.212961, 52.133015]
                              ]]}

min_buck_lat = min(p[1] for p in buckingham['coordinates'][0])
max_buck_lat = max(p[1] for p in buckingham['coordinates'][0])
min_buck_lon = min(p[0] for p in buckingham['coordinates'][0])
max_buck_lon = max(p[0] for p in buckingham['coordinates'][0])

buckingham_centre = [min_buck_lat + (max_buck_lat - min_buck_lat) / 2, 
                     min_buck_lon + (max_buck_lon - min_buck_lon) / 2]

buckingham_region_query = {'loc': {'$geoWithin': {'$geometry': buckingham}}}

In [ ]:
m = folium.Map(buckingham_centre, zoom_start=11)    
add_accidents_markers(m, buckingham_region_query)
m

In [ ]:
def merge_dicts(this, other):
    this_copy = this.copy()
    for k in other:
        this_copy[k] = other[k]
    return this_copy

In [ ]:
m = folium.Map(buckingham_centre, zoom_start=11) 

add_accidents_markers(m, merge_dicts(buckingham_region_query, {'Accident_Severity': 1}),
    fill_color='#ff0000', number_of_sides=5, radius=10, rotation=54)

add_accidents_markers(m, merge_dicts(buckingham_region_query, {'Accident_Severity': 2}),
    fill_color='#ffff00', number_of_sides=4, radius=7, rotation=0)

add_accidents_markers(m, merge_dicts(buckingham_region_query, {'Accident_Severity': 3}),
    fill_color='#00ff00', number_of_sides=3, radius=5, rotation=30)
m

## Activity 4: Colour code accidents by number of vehicles

In [ ]:
milton_keynes = {'type': 'Polygon',
                 'coordinates': [[[-0.877025, 52.092317],
                                  [-0.651709, 52.092317],
                                  [-0.651709, 51.958628],
                                  [-0.877025, 51.958268],
                                  [-0.877025, 52.092317]
                                 ]]}

min_mk_lat = min(p[1] for p in milton_keynes['coordinates'][0])
max_mk_lat = max(p[1] for p in milton_keynes['coordinates'][0])
min_mk_lon = min(p[0] for p in milton_keynes['coordinates'][0])
max_mk_lon = max(p[0] for p in milton_keynes['coordinates'][0])

mk_centre = [min_mk_lat + (max_mk_lat - min_mk_lat) / 2, min_mk_lon + (max_mk_lon - min_mk_lon) / 2]

mk_region_query = {'loc': {'$geoWithin': {'$geometry': milton_keynes}}}

In [ ]:
# Categories used:
#  <=1 vehicles: white
#  2-3 vehicles: green
#  4-6 vehicles: yellow
#  >=7 vehicles: red

m = folium.Map(mk_centre, zoom_start=12) 

add_accidents_markers(m, merge_dicts(mk_region_query, {'Number_of_Vehicles': {'$lte': 1}}),
    fill_color='#ffffff', number_of_sides=9, radius=5, rotation=0)

add_accidents_markers(m, merge_dicts(mk_region_query, {'Number_of_Vehicles': {'$gte': 2, '$lte': 3}}),
    fill_color='#00ff00', number_of_sides=5, radius=5, rotation=54)

add_accidents_markers(m, merge_dicts(mk_region_query, {'Number_of_Vehicles': {'$gte': 4, '$lte': 6}}),
    fill_color='#ffff00', number_of_sides=4, radius=9, rotation=0)

add_accidents_markers(m, merge_dicts(mk_region_query, {'Number_of_Vehicles': {'$gt': 6}}),
    fill_color='#ff0000', number_of_sides=5, radius=11, rotation=54)
m